In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

## The Trigger class

A trigger is a mechanism that determines when a neuron should be fired or activated. It is a conditional check that evaluates certain criteria, such as the state of the neuron's buffer messages or the time elapsed since the last firing. If the condition is met, the trigger returns a list of predecessors that are ready to be fired, otherwise it returns None. The purpose of a trigger is to control the flow of information and activation of neurons in a neural network or other complex system. In the provided examples, we see two types of triggers:

In [2]:
from core.trigger import Trigger

#### Example 1 : A trigger that checks the neuron cache, and fire when all elements of the cache are not empty

In [3]:
class AllPredecessorTrigger(Trigger):
    def condition(self, neuron):
        cond=True
        predecessors = []
        for predecessor, message in neuron.buffer.messages.items():
            predecessors.append(predecessor)
            if message.type=="empty":
                cond=False
        if cond:
            return predecessors ## Here we return a list of predecessors as the condition is met
        return None ## Here we return None as at least one buffer message is empty

#### Example 2 : A trigger that check the clock, and fire if the time since last fire is too big, return all context that are already ready

In [4]:
import time

class CronWithReadyTrigger(Trigger):
    def __init__(self, patience):
        self.patience=patience
        
    def condition(self, neuron):
        if time.time() - neuron.clock.last_fired < self.patience:
            return None ## Returning none as not enough time as elapsed since last fire
        
        cond=True
        predecessors = []
        for predecessor, message in neuron.buffer.messages.items():
            if message.type!="empty":
                predecessors.append(predecessor)
        return predecessors ## Too much time has elapsed, so we collect finished predecessors and fire them

# The Step Class

A Step represents a single unit of processing in a conversation flow. It is responsible for generating a reply based on the conversation context and inputs. A Step is the basic building block of a conversation flow, and it can be used to perform various tasks such as text processing, intent identification, and response generation.

### Creating a Step
To create a Step, you need to follow these steps:

1. **Inherit from the `Step` class**: Create a new class that inherits from the `Step` class. This will allow you to override the `forward` method and implement your custom logic.
2. **Implement the `forward` method**: The `forward` method is where you will implement the logic for processing the conversation context and inputs to generate a reply. This method takes in three parameters:
	* `conversation`: A dictionary containing the conversation data.
	* `context`: A dictionary containing the conversation context.
	* `working_memory`: An instance of the `WorkingMemory` class, which is used to store and retrieve information during the conversation.
3. **Define the `__init__` method**: The `__init__` method is used to initialize the Step instance. You can use this method to set up any necessary attributes or parameters for the Step.
4. **Call the `super().__init__` method**: In the `__init__` method, you should call the `super().__init__` method to ensure that the parent class is properly initialized.


#### Example: Call a llm with the neuron context in the past conversation, and a custom prompt as last user message

In [5]:
from typing import Dict, Any
from core.message import Message, Reply
from core.st_memory import WorkingMemory
from core.step import Step

In [6]:
# Creating a Step instance that run an llm on the conversation + context
def call_llm(messages, call_args):
    return "ran"

class LLMStep(Step):
    def __init__(self, str_input: str= None, step_args: Dict[str, Any] = None):
        super().__init__(str_input, step_args)
        self.str_input=str_input
        self.step_args=step_args
        
    def forward(self, conversation: Dict[str, Any]=[], context: Dict[str, Message]={}, working_memory: WorkingMemory = None) -> Reply:
        messages=conversation.copy()

        step_context=[]
        for src_id, msg in context.items():
            step_context.append(src_id)
            messages.append({'role':'user','content':msg.reply.str_input})
            messages.append({'role':'assistant','content':msg.reply.str_output})
        
        messages.append({'role':'user','content':self.str_input})
        
        reply = Reply(
            str_input=self.str_input,
            str_output=call_llm(messages, call_args=self.step_args),
            context=step_context
        )
        return reply

step = LLMStep("What is love")

step.forward()

Reply(str_input='What is love', str_output='ran', context=[], metadata={})

## The Message Class

### Definition of a Message
A Message represents a core data structure in the system, used to exchange information between neurons. The Message model includes attributes such as type, sender and receiver IDs, reply, priority, creation time, and metadata.

### Components of a Message
A Message consists of the following components:

1. **MessageType**: An Enum representing the types of messages that can be sent, including `EMPTY`, `PERSISTENT`, and `PAYLOAD`.
2. **Reply**: A model representing the response to a message, containing inputs, outputs, and metadata.
3. **Message Type**: The type of the message, which can be one of the MessageType Enum values.
4. **Sender and Receiver IDs**: The IDs of the sender and receiver of the message.
5. **Priority**: The priority of the message, which can range from 0 to 10.
6. **Creation Time**: The time the message was created, which is automatically set to the current time.
7. **Metadata**: Additional metadata associated with the message.

Message can be of three types:
- empty : the message will replace any existing message in the target buffer
- payload : the message contains a reply, and will take its place in the target buffer, but it will be cleared from it when the target will fire
- persistant : same as the payload, but stays in the target buffer on fire, it can be replaced by an empty message or a payload message though, or cleaned if the buffer have an expiration.

In [7]:
from core.message import Message, Reply

empty_message=Message(type="empty", sender_id="node1", receiver_id="node2")
payload_message=Message(type="payload", sender_id="node1", receiver_id="node2", reply=Reply(str_input='Hello there', str_output='Ran'))
payload_message=Message(type="persistent", sender_id="node1", receiver_id="node2", reply=Reply(str_input='Hello there', str_output='Ran'))

## The Activation Class

An Activation represents a mechanism for routing replies to neuron successors in a neural network. The Activation class is an abstract base class that provides a blueprint for routing replies to neuron successors.

### Creating an Activation Function
To create an Activation Function, you need to follow these steps:

1. **Inherit from the `Activation` class**: Create a new class that inherits from the `Activation` class. This will allow you to override the `route` method and implement your custom logic.
2. **Implement the `route` method**: The `route` method is where you will implement the logic for routing a reply to the successors of a neuron. This method takes in three parameters:
	* `reply`: The reply to be routed.
	* `neuron`: The neuron that the reply is being routed from.
	* `working_memory`: An instance of the `WorkingMemory` class, which is used to store and retrieve information during the conversation.
3. **Define the routing logic**: In the `route` method, you need to define the logic for routing the reply to the successors of the neuron. This can involve creating a list of messages to be sent to the successors.

#### Example 1 : Pass the message as a standard message to all successors

In [13]:
from core.activation import Activation
from core.neuron import Neuron
import random

class FullActivation(Activation):
    def route(self, reply:Reply, neuron:Neuron, working_memory=None):
        routed_messages=[]

        for successor, successor_description in neuron.successors.items():
            routed_messages.append(Message(type="payload", sender_id=neuron.id, receiver_id=successor, reply=reply))
            
        return routed_messages

#### Example 2 : 25% chance on each kind of message
- 25% chances to get a normal message
- 25% chances to get a persistant message
- 25% chances to get a reset
- 25% chances to receive nothing

In [14]:
from core.activation import Activation
from core.neuron import Neuron
import random

class FiftyFiftyActivation(Activation):
    def route(self, reply:Reply, neuron:Neuron, working_memory=None):
        routed_messages=[]

        for successor, successor_description in neuron.successors.items():
            r=random.uniform(0,1)
            if r < 0.25:
                routed_messages.append(Message(type="payload", sender_id=neuron.id, receiver_id=successor, reply=reply))
            elif r < 0.5:
                routed_messages.append(Message(type="persistent", sender_id=neuron.id, receiver_id=successor, reply=reply))
            elif r < 0.75:
                routed_messages.append(Message(type="empty", sender_id=neuron.id, receiver_id=successor)) ## No need for reply here
            else:
                pass
        return routed_messages

Obviously, you can design smarter conditions on the successor, and even try an llm condition

In [15]:
from core.activation import Activation
from core.neuron import Neuron
import random

def llm_classifier(description):
    if len(description) < 10:
        return True
    else:
        return False
    

class DummyLLMActivation(Activation):
    def route(self, reply:Reply, neuron:Neuron, working_memory=None):
        routed_messages=[]

        for successor, successor_description in neuron.successors.items():
            cond=llm_classifier(successor_description)
            if cond:
                routed_messages.append(Message(type="payload", sender_id=neuron.id, receiver_id=successor, reply=reply))
        return routed_messages

## The neuron Class

### Definition of a Neuron
A Neuron represents a node in a network, responsible for processing and transmitting information. The Neuron class has methods to fire the neuron, handle retries, and check if the neuron should fire.

### Components of a Neuron
A Neuron consists of the following components:

1. **ID**: A unique identifier for the neuron.
2. **Predecessors**: A list of predecessor neuron IDs.
3. **Successors**: A dictionary of successor neuron IDs and their descriptions.
4. **Triggers**: A list of triggers that determine when the neuron should fire.
5. **Step Function**: A function that processes the input messages and returns a reply.
6. **Activation Function**: A function that maps the reply to the successor neurons.
7. **Buffer**: A buffer that stores messages from predecessor neurons.
8. **Clock**: A clock that tracks the time and updates the neuron's state.

```python
neuron = Neuron(
    id="neuron_1", ## Unique id of the neuron in the brain
    description="the description of the node",
    predecessors={"predecessor_1":"description_1", "predecessor_2":"description_2"}, ## List of predecessors of the neuron
    successors={"successor_1": "description_1", "successor_2": "description_2"}, ## List of successors of the neuron
    triggers=[trigger_1, trigger_2], ## list of triggers. Be careful to the order, if two trigger are activated at the same time, only the first will fire the neuron
    step=step_function, ## The step that is applied after the trigger
    activation=activation_function, ## The activation class
    is_entrypoint=True, ## If the neuron is an entrypoint. If this is the case, it will fire as soon as the brain starts, without any other context than the conversation. A brain must have at least one entypoint neuron
    is_terminal=False, ## If the neuron is terminal. If this is the case, the brain will stop after the neuron fire. A brain can have no terminal node and run until timeout.
)
```

#### Example 1:

In [28]:
import time

# Define the trigger, step, activation, and neuron
trigger = CronWithReadyTrigger(patience=1)
step = LLMStep("What is love?")
activation = FiftyFiftyActivation()
description = "Answer to the timely question: what is love"

predecessors = {
    'node1': 'a predecessor and successor node',
    'node2': 'a predecessor node',
}

successors = {
    'node3': 'a successor node',
    'node1': 'a predecessor and successor node',
}

neuron = Neuron(
    id="node4",
    description=description,
    predecessors=predecessors,
    successors=successors,
    triggers=[trigger],  # we use only one trigger here
    step=step,
    activation=activation,
    is_entrypoint=False,
    is_terminal=True
)

# Update the content of node1
node1_reply = Reply(str_input='Hello there', str_output='Ran')
node1_message = Message(type="payload", sender_id="node1", receiver_id='node4', reply=node1_reply)
neuron.receive(node1_message)

# Print the buffer messages
print("Buffer Messages:")
for message in neuron.buffer.messages:
    print(message)

# Print the last fired time
last_fired = neuron.clock.last_fired
print(f"\nLast Fired: {last_fired}")

# Check if the neuron should fire
should_fire = neuron.should_fire()
print(f"\nShould Fire: {should_fire}")

# Wait for 2 seconds
print("\nWaiting for 2 seconds...")
time.sleep(2)

# Check if the neuron should fire again
context = neuron.should_fire()
print(f"\nContext to Trigger: {context}")

# Fire the neuron
conversation = []
neuron, reply, messages = neuron.fire(conversation, context)

# Print the reply and messages
print(f"\nReply: {reply}")
print("\nMessages:")
print(messages)

Buffer Messages:
node1
node2

Last Fired: None

Should Fire: None

Waiting for 2 seconds...

Context to Trigger: ['node1']

Reply: str_input='What is love?' str_output='ran' context=['node1'] metadata={}

Messages:
[Message(type=<MessageType.PERSISTENT: 'persistent'>, sender_id='node4', receiver_id='node3', reply=Reply(str_input='What is love?', str_output='ran', context=['node1'], metadata={}), priority=0, created_at=1736713498.3378732, metadata={}), Message(type=<MessageType.PERSISTENT: 'persistent'>, sender_id='node4', receiver_id='node1', reply=Reply(str_input='What is love?', str_output='ran', context=['node1'], metadata={}), priority=0, created_at=1736713498.337876, metadata={})]


## The Brain class enables you to orchestrate several neurons.

### Definition of a Brain
A Brain represents a neural network that can execute conversations. The Brain class has methods to run the brain execution synchronously and yield the list of messages each time a process is finished.

### Components of a Brain
A Brain consists of the following components:

1. **Name**: A unique name for the brain.
2. **Neurons**: A list of neurons in the brain, where each neuron is an instance of the `Neuron` class.
3. **Working Memory**: A working memory that stores messages and has a maximum size and overflow strategy.
4. **Workers**: The number of worker threads used to execute the brain.
5. **Timeout**: The timeout for the brain execution.
6. **Iteration Delay**: The delay between iterations.
7. **Max Concurrent Fires**: The maximum number of concurrent fires.

### Creating a Brain
To create a Brain, you can use the `Brain` class and provide the required attributes. For example:
```python
brain = Brain(
    name="my_brain",
    neurons=[neuron_1, neuron_2, neuron_3],
    max_working_memory_size=10000,
    working_memory_overflow_strategy="fifo",
    workers=8,
    timeout=120,
    iteration_delay=0.5,
    max_concurrent_fires=100,
)
```

### Running a Brain
The `run` method is used to run the brain execution synchronously. It takes a conversation as input and yields the list of messages each time a process is finished. The conversation is a list of dictionaries, where each dictionary represents a message with a role and content.
